# Cardiotocographic Classification using TensorFlow and Keras
##### Cardiotocographic classification for fetal heart-rate and uterine contractions, implemented with a PySpark Pipeline

Dataset from the UCI data repository: https://archive.ics.uci.edu/ml/datasets/cardiotocography

In [1]:
#Pipeline dependencies
from pyspark.ml import Pipeline
from pyspark.ml.classification import LogisticRegression
from pyspark.ml.feature import HashingTF, Tokenizer
from pyspark.sql import SparkSession

#Data manipulation, analysis and plotting tools
import pandas as pd
import numpy as np
import matplotlib as plt

#Machine Learning libraries
import sklearn as sk
import tensorflow as tf

In [2]:
#Define the session and cluster
spark = SparkSession.builder \
                    .master('local[4]') \
                    .appName('cardiotocography_pipeline') \
                    .getOrCreate()